<a href="https://colab.research.google.com/github/Hibi1001/practice/blob/main/section_2/03_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 演習
機械学習のアルゴリズム「サポートベクターマシン」 を使い、タイタニック号の生存者を予測しましょう。  
サポートベクターマシンのモデルを作成し、訓練するコードを記述してください。

## データの読み込み


In [19]:
import numpy as np
import pandas as pd

train_data = pd.read_csv("train-2.csv")  # 訓練データ
test_data = pd.read_csv("test-2.csv") # テストデータ

train_data.head()
#test_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## データの前処理

In [20]:
test_id = test_data["PassengerId"]  # 結果の提出時に使用

labels = ["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]
train_data = train_data[labels + ["Survived"]]
test_data = test_data[labels]

train_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,male,22.0,1,0,7.2500,S,0
1,1,female,38.0,1,0,71.2833,C,1
2,3,female,26.0,0,0,7.9250,S,1
3,1,female,35.0,1,0,53.1000,S,1
4,3,male,35.0,0,0,8.0500,S,0


In [21]:
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
 7   Survived  891 non-null    int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       332 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      417 non-null    float64
 6   Embarked  418 non-null

Age, Embarked, Fareの三つの項目に欠損値が確認される\
fillna()を用いて欠損値の補完をする

In [22]:
# Age
age_mean = train_data["Age"].mean()  # 平均値
train_data["Age"] = train_data["Age"].fillna(age_mean)
test_data["Age"] = test_data["Age"].fillna(age_mean)

# Fare
fare_mean = train_data["Fare"].mean()  # 平均値
train_data["Fare"] = train_data["Fare"].fillna(fare_mean)
test_data["Fare"] = test_data["Fare"].fillna(fare_mean)

# Embarked
embarked_mode = train_data["Embarked"].mode()  # 最頻値
train_data["Embarked"] = train_data["Embarked"].fillna(embarked_mode)
test_data["Embarked"] = test_data["Embarked"].fillna(embarked_mode)

In [23]:
cat_labels = ["Sex", "Pclass", "Embarked"]  # カテゴリ変数のラベル
train_data = pd.get_dummies(train_data, dtype = int,  columns=cat_labels)
test_data = pd.get_dummies(test_data, dtype = int, columns=cat_labels)

train_data.head()

,Age,SibSp,Parch,Fare,Survived,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,22.0,1,0,7.2500,0,0,1,0,0,1,0,0,1
1,38.0,1,0,71.2833,1,1,0,1,0,0,1,0,0
2,26.0,0,0,7.9250,1,1,0,0,0,1,0,0,1
3,35.0,1,0,53.1000,1,1,0,1,0,0,0,0,1
4,35.0,0,0,8.0500,0,0,1,0,0,1,0,0,1


*f(x) = t\
みたいなノリなのかな？\
trainデータの入力（x）からtを予測する\
x_trainには正解の列(Survived)を取り除いたものを入れる

## モデルの訓練

In [24]:
t_train = train_data["Survived"]  # 正解
x_train = train_data.drop(labels=["Survived"], axis=1)  # "Survived"の列を削除して入力に
x_test = test_data

サポートベクターマシン（Support Vector Machine、SVM）とは、パターン識別のための教師あり機械学習のアルゴリズムです。  
「マージン最大化」というアイディアに基づいているのですが、しばしば優れたパターン識別能力を発揮します。

以下のセルに、サポートベクターマシンのモデルを作成し、訓練するコードを記述してください。  
以下の、scikit-learnの公式ドキュメントなどが参考になります。  
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [26]:
from sklearn.svm import SVC  # SVC: Support Vector Classification

model =  SVC(kernel = "rbf",  C=1.0, gamma='scale')# ←ここにモデルを作成するコードを記述する
model.fit(x_train, t_train)
  # ←ここにモデルを訓練するコードを記述する

SVC()

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets

## 結果の提出

テストデータを使って予測を行います。  
予測結果は、形式を整えた上で提出用のcsvファイルとして保存します。

In [32]:
# 判定
y_test = model.predict(x_test)

# 形式を整える
survived_test = pd.Series(y_test, name="Survived")
subm_data = pd.concat([test_id, survived_test], axis=1)

# 提出用のcsvファイルを保存
subm_data.to_csv("submission_titanic.csv", index=False)

subm_data

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


## 解答例
以下は解答例です。

In [ ]:
from sklearn.svm import SVC  # SVC: Support Vector Classification

model = SVC()  # ←ここにコードを記述する
model.fit(x_train, t_train)  # ←ここにコードを記述する